In [1]:
import requests
import json

In [2]:
class Patients:
    def __init__(self, pid, type, value,reference):
        self.pid = pid
        self.type = type
        self.value = value
        self.ref = reference

    def __str__(self):
        return "pid:{}, type:{},value: {}".format(self.pid,self.type,self.value)

In [6]:
OAuthurl = 'https://sandbox86.tactiorpm7000.com/token.php'
APIurl = 'https://sandbox86.tactiorpm7000.com/tactio-clinician-api/1.1.4'

# '''Authentication'''
body = {
"grant_type": "password",
"client_id": "083e9a44a763473fbeb62fbf90b74551",
"client_secret": "ba09798f0921456e8b4e5e4588ea536d",
"username": "tactioClinician",
"password": "tactio"
}

response = requests.post(OAuthurl, data=body)
auth_code = response.json()["access_token"]

In [30]:
# initialize values (patients_list == type.dict)
patient_dict = {}

### add BMI to patient_dict

In [52]:
observation_param = [("code", "39156-5"),("code", "39156-5")]
patient_observation_raw = requests.get(APIurl + '/Observation', headers={
    'Authorization': 'Bearer {}'.format(auth_code)}, params=observation_param)
patient_obs_json = patient_observation_raw.json()

for patient in patient_obs_json['entry']:
    # get the patient name
    patient_name = patient['resource']['subject']['reference']
    # if patient_dict does not have the name, create a empty key
    if (patient_name not in patient_dict):
        patient_dict[patient_name] = {}
    # add bmi to key
    patient_dict[patient_name]['bmi'] = patient['resource']['valueQuantity']['value']

### add Fat Ratio to patient_dict

In [58]:
observation_param = [("code", "73708-0"),("code", "73708-0")]
patient_observation_raw = requests.get(APIurl + '/Observation', headers={
    'Authorization': 'Bearer {}'.format(auth_code)}, params=observation_param)
patient_obs_json = patient_observation_raw.json()

for patient in patient_obs_json['entry']:
    # get the patient name
    patient_name = patient['resource']['subject']['reference']
    # if patient_dict does not have the name, create a empty key
    if (patient_name not in patient_dict):
        patient_dict[patient_name] = {}
    # add bmi to key
    patient_dict[patient_name]['fat_ratio'] = patient['resource']['valueQuantity']['value']

In [59]:
patient_dict

{'Patient/e5850a8f-5a21-49c5-80be-5270e3b126bb': {'bmi': '23.1279',
  'fat_ratio': '17.1673'},
 'Patient/3ed878a0-60aa-48c4-bb52-d64c72c6ef75': {'bmi': '22.0026',
  'fat_ratio': '23.8395'},
 'Patient/8e4955b4-7313-451f-abc1-af165af6f080': {'bmi': '20.5519',
  'fat_ratio': '24.5438'},
 'Patient/eef20546-a9a9-4eea-b6ab-17fbd8752f25': {'bmi': '41.8204'},
 'Patient/dd62061c-4475-4398-8569-fa0e12f62b33': {'bmi': '53.8119'},
 'Patient/15cb54b1-7983-40c7-91af-43701984d90b': {'bmi': '77.2759'},
 'Patient/cb835631-fef0-4d5c-9610-fd0b7882ea37': {'bmi': '23.8793',
  'fat_ratio': '15.1644'},
 'Patient/71c67b49-30eb-4711-855d-a0d5f3c01cfd': {'bmi': '23.6645',
  'fat_ratio': '19.3192'},
 'Patient/85fb9c43-23ff-4f95-9165-56e7919eb300': {'bmi': '23.3489',
  'fat_ratio': '13.5303'},
 'Patient/d5570329-3ac9-4200-bfd3-f0993c3e5d4d': {'bmi': '23.2726',
  'fat_ratio': '12.6996'},
 'Patient/5a52916d-4c45-41de-b707-0c79b4482b39': {'bmi': '21.6746',
  'fat_ratio': '25.9025'},
 'Patient/39dd6a84-4f66-44e2-8d61